### Setup variables, paths, and create artifact registry

In [2]:
!pwd

/home/jupyter/vertex-workbench-custom-images


In [4]:
REPO_NAME="workbench"
REGION="us-central1"
PROJECT="hybrid-vertex" # TODO: update with your project_id
IMAGE_ID="python-3.9"
DERIVED_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPO_NAME}/derived-{IMAGE_ID}"
SCRATCH_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPO_NAME}/scratch-{IMAGE_ID}"

DERIVED_CONTAINER="derived_container"
SCRATCH_CONTAINER="scratch_container"

!mkdir {DERIVED_CONTAINER} {SCRATCH_CONTAINER}

!gcloud beta artifacts repositories create {REPO_NAME} \
    --repository-format=docker \
    --location=$REGION

In [5]:
!gcloud config set project $PROJECT

Updated property [core/project].


In [15]:
# !gcloud projects add-iam-policy-binding hybrid-vertex --member=serviceAccount:934903580331-compute@developer.gserviceaccount.com --role=roles/artifactregistry.admin

In [6]:
!gcloud auth configure-docker us-central1-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
Docker configuration file updated.


### Create Docker image derived from Deeplearning containers

In [7]:
%%writefile {DERIVED_CONTAINER}/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf-gpu:latest
RUN conda create -n python-3.9
RUN conda install -n python-3.9 ipykernel python=3.9 
#SHELL ["/bin/bash", "-c"]
#RUN source /opt/conda/bin/activate /opt/conda/envs/python-3.9 && python -m ipykernel install --user --name=python-3.9 && conda deactivate


Writing derived_container/Dockerfile


In [8]:
!docker build . -f $DERIVED_CONTAINER/Dockerfile -t $DERIVED_IMAGE_NAME
!docker push $DERIVED_IMAGE_NAME

Sending build context to Docker daemon  144.9kB
Step 1/3 : FROM gcr.io/deeplearning-platform-release/tf-gpu:latest
latest: Pulling from deeplearning-platform-release/tf-gpu

17ec1767: Pulling fs layer 
43e57386: Pulling fs layer 
04926279: Pulling fs layer 
b7509e42: Pulling fs layer 
db7c1fa2: Pulling fs layer 
d320cf87: Pulling fs layer 
fc7ee50b: Pulling fs layer 
caf93b3f: Pulling fs layer 
7dd23c11: Pulling fs layer 
3345b975: Pulling fs layer 
791d4ecf: Pulling fs layer 
b700ef54: Pulling fs layer 
48849d70: Pulling fs layer 
87af4bab: Pulling fs layer 
64f0f12e: Pulling fs layer 
55789b0d: Pulling fs layer 
1f324c40: Pulling fs layer 
f0629bf3: Pulling fs layer 
243fb034: Pulling fs layer 
b78c8efc: Pulling fs layer 
2b0c405e: Pulling fs layer 
9b85dfbe: Pulling fs layer 
5ce2bd1c: Pulling fs layer 
5ce2bd1c: Waiting fs layer 
4546170a: Waiting fs layer 
28a4b0a0: Pulling fs layer 
eedfcad6: Pulling fs layer 
ad75b4a9: Waiting fs layer 
3bc84913: Pulling fs layer 
6b7d44fa: Pull

### Docker Image from scratch

In [9]:
%%writefile {SCRATCH_CONTAINER}/jupyter_notebook_config.py
c.NotebookApp.ip = '*'
c.NotebookApp.token = ''
c.NotebookApp.password = ''
c.NotebookApp.open_browser = False
c.NotebookApp.port = 8080
c.NotebookApp.terminado_settings = {'shell_command': ['/bin/bash']}
c.NotebookApp.allow_origin_pat = (
'(^https://8080-dot-[0-9]+-dot-devshell\.appspot\.com$)|'
'(^https://colab\.research\.google\.com$)|'
'((https?://)?[0-9a-z]+-dot-(?:us|asia|europe|northamerica|southamerica)-?[0-9a-z]+\.notebooks\.googleusercontent.com)')
c.NotebookApp.allow_remote_access = True
c.NotebookApp.disable_check_xsrf = False

Writing scratch_container/jupyter_notebook_config.py


In [10]:
%%writefile {SCRATCH_CONTAINER}/Dockerfile
FROM ubuntu:20.04

## for apt to be noninteractive
ENV DEBIAN_FRONTEND noninteractive
ENV DEBCONF_NONINTERACTIVE_SEEN true

RUN apt-get update && apt-get install -y \
    python3.9 \
    python3-pip

# Use this to set the default python to 3.9
# RUN sudo update-alternatives  --set python /usr/bin/python3.9
RUN update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
RUN update-alternatives --set python3 /usr/bin/python3.9

RUN pip install jupyter -U && pip install jupyterlab

# RUN python3.9 -m pip install ipykernel && python3.9 -m ipykernel install --user --name "python-3.9"

EXPOSE 8080
             
RUN mkdir /root/.jupyter
             
COPY scratch_container/jupyter_notebook_config.py /root/.jupyter

ENV pwd=""

ENTRYPOINT exec jupyter-lab --ip=0.0.0.0 --port=8080 --no-browser --allow-root --ServerApp.allow_origin="*" --NotebookApp.token="$pwd" --NotebookApp.password="$pwd"

Writing scratch_container/Dockerfile


In [11]:
!docker build . -f $SCRATCH_CONTAINER/Dockerfile -t $SCRATCH_IMAGE_NAME
!docker push $SCRATCH_IMAGE_NAME

Sending build context to Docker daemon  210.4kB
Step 1/12 : FROM ubuntu:20.04
20.04: Pulling from library/ubuntu

Digest: sha256:fd92c36d3cb9b1d027c4d2a72c6bf0125da82425fc2ca37c414d4f010180dc19[1A
Status: Downloaded newer image for ubuntu:20.04
 ---> 20fffa419e3a
Step 2/12 : ENV DEBIAN_FRONTEND noninteractive
 ---> Running in caa5e8ee52e0
Removing intermediate container caa5e8ee52e0
 ---> b7ff878a7ab7
Step 3/12 : ENV DEBCONF_NONINTERACTIVE_SEEN true
 ---> Running in 2b883dc19b0c
Removing intermediate container 2b883dc19b0c
 ---> 2913f524ebea
Step 4/12 : RUN apt-get update && apt-get install -y     python3.9     python3-pip
 ---> Running in eb158cf050ef
Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/multiverse